In [16]:
from kth_dataset import KTH_Dataset
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from LSTMscratch import LSTMscratch
import torch
import numpy as np
import torch.nn as nn
from Model_Wrapper import Wrapper
import tqdm
import helper 


In [17]:
helper.set_random_seed()


Random seed set to: 13


In [18]:
# Create a dataset and a DataLoader
transforms = {"train":transforms.Compose([
        transforms.ToTensor(),  # Convert image to tensor
        transforms.Resize([32,32]),
    ]),
        "val": transforms.Compose([
        transforms.ToTensor(),  # Convert image to tensor
        transforms.Resize([32,32]),
    ]) }
# Initialize dataset
test_dataset = KTH_Dataset(root_dir="/home/nfs/inf6/data/datasets/kth_actions/processed", sequence_length=15,split="test", transform=transforms["train"],use_saved_samples=False)
train_dataset = KTH_Dataset(root_dir="/home/nfs/inf6/data/datasets/kth_actions/processed", sequence_length=15,split="train", transform=transforms["val"],use_saved_samples=False)


In [19]:
train_dataset.__len__()

181826

In [20]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True,num_workers=8,pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False,num_workers=8,pin_memory=True)
print("test_dataset per epoch",next(iter(test_loader))[0].size())

test_dataset per epoch torch.Size([16, 15, 1, 32, 32])


In [21]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [22]:
model = LSTMscratch(input_dim=64, hidden_dim=16, number_of_layers=3, device=device)

In [23]:
# model_t = SequentialClassifierWithCells(emb_dim=128, hidden_dim=128, num_layers=2, mode="zeros")
model = model.to(device)


In [24]:
# classification loss function
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

# Decay LR by a factor of 0.1 every 5 epochs
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.2)

In [25]:
device

device(type='cuda')

In [26]:
model = model.to(device)

In [27]:
classes = ["Boxing","HandClapping","HandWaving","Jogging","Running","Walking"]
# model = SequentialClassifierWithCells(emb_dim=128, hidden_dim=128, num_layers=2, mode="zeros").to(device)
writer = helper.new_writer("models","LSTMscratch")
train = Wrapper(model_name="LSTMscratch",model = model, device = device, criterion = criterion, optimizer = optimizer,writer=writer,show_progress_bar= True)
train.train(10,train_loader,test_loader,classes)
writer.close()
train.valid_accuracy()
train.plot_loss_acc()


Epoch 1 Iter 11365: loss 0.99723.: 100%|██████████| 11365/11365 [09:01<00:00, 20.98it/s]

Accuracy/train 32.661569900610296
Loss/train 1.3128931049675445


Accuracy/test 37.566328991799324
Loss/test 1.2995702980089503


Epoch 2 Iter 11365: loss 0.75196.: 100%|██████████| 11365/11365 [09:06<00:00, 20.80it/s]

Accuracy/train 44.014288752196606
Loss/train 1.1257753634054233


...

Epoch 3 Iter 11365: loss 1.04620.: 100%|██████████| 11365/11365 [09:08<00:00, 20.73it/s]

Accuracy/train 48.04004279177489
Loss/train 1.0560457759841235


Accuracy/test 38.51201961730182
Loss/test 1.6690083270355482


Epoch 4 Iter 2981: loss 1.02389.:  26%|██▌       | 2982/11365 [02:22<06:41, 20.87it/s]


KeyboardInterrupt: 